In [1]:
import psycopg2
hostname = 'localhost'
username = 'postgres'
password = 'pass'
database = 'pgroute'

/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
myConnection = psycopg2.connect(host=hostname, user=username, password=password, dbname=database )

In [3]:
# Simple routine to run a query on a database and print the results:
def doQuery( conn, query ) :
    cur = conn.cursor()
    cur.execute(query)
    if cur.rowcount>0:
        return cur.fetchall()
    cur.close()

    
def doQueryv2( conn, query ) :
    cur = conn.cursor()
    cur.execute(query)
    cur.close()

In [4]:
long_s, lat_s = (80.2049,13.1090) ##VILLIVAKAM
long_t, lat_t = (80.1275, 12.9229) ##TAMBARAM

In [5]:
s_geom = 'POINT({} {})'.format(long_s, lat_s)
t_geom = 'POINT({} {})'.format(long_t, lat_t)

In [6]:
s_query = "SELECT source FROM ways WHERE source_osm in (SELECT osm_id FROM ways_vertices_pgr ORDER BY the_geom <-> ST_GeometryFromText('{}',4326) LIMIT 1) LIMIT 1".format(s_geom)
source = doQuery( myConnection, s_query )
t_query =  "SELECT target FROM ways WHERE target_osm in (SELECT osm_id FROM ways_vertices_pgr ORDER BY the_geom <-> ST_GeometryFromText('{}',4326) LIMIT 1) LIMIT 1 ".format(t_geom)
target = doQuery( myConnection, t_query )

In [7]:
print(source, target)

[(23874,)] [(34997,)]


In [8]:
length_query = "SELECT osm_id,lon,lat FROM ways_vertices_pgr ORDER BY the_geom <-> ST_GeometryFromText('{}',4326) ASC".format(s_geom)

In [9]:
dist_query = "SELECT * INTO DD FROM pgr_dijkstra('SELECT gid AS id, source, target, length AS cost FROM ways',{}, {},directed := false)".format(source[0][0], target[0][0])
doQueryv2( myConnection, dist_query )

In [10]:
length_query = "SELECT SUM(length_m) FROM ways WHERE gid IN (SELECT edge FROM DD)"
doQuery( myConnection, length_query) # GETTING TOTAL DISTANCE

[(26409.3479128543,)]

In [11]:
pt_query = "SELECT node FROM DD;"
dd = doQuery( myConnection, pt_query)

In [12]:
dd[:10]

[(23874,),
 (16783,),
 (54298,),
 (29189,),
 (22196,),
 (4314,),
 (8861,),
 (50277,),
 (57313,),
 (31506,)]

In [13]:
pt_query = "SELECT ST_AsText(the_geom),id FROM ways_vertices_pgr WHERE id IN (SELECT node FROM DD)"
points = doQuery( myConnection, pt_query)

In [14]:
points[:4]

[('POINT(80.1298969 12.9270667)', 179),
 ('POINT(80.1301152 12.9286012)', 222),
 ('POINT(80.2050864 13.0161323)', 256),
 ('POINT(80.2114242 13.0577018)', 434)]

In [15]:
import pandas as pd
path = pd.DataFrame(dd)
point = pd.DataFrame(points)

In [16]:
point.head()

,0,1
0,POINT(80.1298969 12.9270667),179
1,POINT(80.1301152 12.9286012),222
2,POINT(80.2050864 13.0161323),256
3,POINT(80.2114242 13.0577018),434
4,POINT(80.2006035 13.0915645),565


In [17]:
point.columns = ['geom', 'node']
path.columns = ['node']

In [18]:
points = list(pd.merge(path,point, how='left', on='node')['geom'])

In [19]:
location = []
for i in range(len(points)):
    temp = list(map(float,points[i].split('(')[1].split(')')[0].split(' ')))
    location.append(temp[::-1])

In [20]:
location[:10]

[[13.1090389, 80.2049354],
 [13.1088743, 80.2049774],
 [13.1076363, 80.2048111],
 [13.1049064, 80.2041766],
 [13.104864, 80.2043249],
 [13.1045994, 80.2043187],
 [13.1038188, 80.2043294],
 [13.1036529, 80.2030155],
 [13.1036318, 80.202886],
 [13.1035379, 80.2021067]]

In [24]:
center = location[int(len(location)/2)]

In [26]:
so = location[0]
target = location[-1]

In [27]:
from ipyleaflet import Map, Polyline, Marker

line = Polyline(
    locations = location,
    color = "green" ,
    )
m = Map(center = center, zoom =12)
marker = Marker(location=target, draggable=False, title='Dwarka')
m.add_layer(marker);
marker = Marker(location= so, draggable=False, title='Sarojini Market')
m.add_layer(marker);
m.add_layer(line)
m


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### STATIC IMAGE

![alt text](https://i.ibb.co/2dGQkYx/Screen-Shot-2019-01-18-at-4-15-56-PM.png)
